In [1]:
from utils import profiler, reader
import numpy as np
from typing import List

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
#Pull in the data
datafile = "../data/day2_input.txt"
#since we will be doing this frequently, I created a module to do this
data = reader.read_from_file(datafile)
data = [x.rstrip().split() for x in data]
data[:10]

[['1', '3', '4', '5', '8', '10', '7'],
 ['48', '51', '54', '55', '55'],
 ['7', '9', '12', '15', '16', '17', '20', '24'],
 ['49', '52', '55', '57', '58', '60', '65'],
 ['73', '75', '77', '80', '81', '78', '81', '82'],
 ['83', '84', '87', '85', '86', '84'],
 ['76', '77', '80', '79', '80', '80'],
 ['25', '27', '28', '29', '28', '30', '34'],
 ['21', '23', '26', '28', '26', '27', '34'],
 ['71', '72', '72', '73', '74', '75', '77', '80']]

# Part 1

### Overview

We are tasked with analyzing lines individually. Each line should be monotonic and adjacent numbers can differ by at most 3. We need to count the number of rows that meet such conditions

### Approach

We can scan through and complete these checks in one pass by seeing whether the levels increase or decrease between first and second element AND whether the difference is at most 3 for all remaining elements. This will scan all the elements in quadratic time, which is the best possible. This might be done with vectorized operations at the cost of some memory by making extra arrays with size len(report) + 1. Padding out the array like this enables us to offset and calculate the differences and use a numpy mask to evaluate whether the values of that difference fall into the conditional, for instance np.any(np.where(0 >= diffs > 3, "Safe", "Unsafe") == "Unsafe"). However, because each report is small, it's not worth the overhead cost.

In [10]:
@profiler.profile
def part1_numpy(data: List) -> int:
    data = [list(map(int, x)) for x in data]
    safe = set()
    
    for j, report in enumerate(data):
        if report[0] < report[1]: #The list must be increasing
            row = np.array(report + [0])
            row_offset = np.array([0] + report)
            diffs = (row - row_offset)[1:-1] #remove first and last element generated by padding
            #the row is only safe if all elements of diffs are greater than 0 and less than or equal to 3
            nonpositive = diffs > np.zeros(diffs.shape)
            within_3 = diffs <= np.full(diffs.shape, 3)
            if nonpositive.all() and within_3.all():
                safe.add(j)
        elif report[0] > report[1]: #The list must be decreasing
            row = np.array(report + [0])
            row_offset = np.array([0] + report)
            diffs = (row_offset - row)[1:-1] #remove first and last element generated by padding
            #the row is only safe if all elements of diffs are greater than 0 and less than or equal to 3
            nonpositive = diffs > np.zeros(diffs.shape)
            within_3 = diffs <= np.full(diffs.shape, 3)
            if nonpositive.all() and within_3.all():
                safe.add(j)

    return len(safe)

part1_numpy(data)

Calling part1_numpy: Memory used 1105920 kB; Execution Time: 0.07770816702395678 s


220

In [19]:
@profiler.profile
def part1(data: List) -> int:
    def check_row(row: List) -> bool:
        if row[0] > row[1]: #List is decreasing
            for i in range(len(row) - 1):
                diff = row[i] - row[i+1]
                if diff <= 0 or diff > 3:
                    return False
        elif row[1] > row[0]: 
            for i in range(len(row) - 1):
                diff = row[i+1] - row[i]
                if diff <= 0 or diff > 3:
                    return False                
        else:
            return False
        return True
                   
    valids = [check_row(list(map(int, row))) for row in data]
    return valids 

result = part1(data)
print(sum(result))


Calling part1: Memory used 1245184 kB; Execution Time: 0.006052749697118998 s
220


### Notes
As you can see from the profiler decorator, the numpy implementation is NOT worth it! It uses much more memory and is about 10 times slower due to the overhead. 

# Part 2

### Overview

Now we want to see how many of these reports can be fixed by removing only one element.

### Approach

I will use a similar approach to before. Modifying the code by packaging up the code that checks each row and rerunning it for all iterations of the row with an element deleted! This will run in cubic time, but because each report itself is small, this will not be a problem

In [18]:
sum(a)

220

In [20]:
@profiler.profile
def part2(data: List) -> int:
    valid = set()

    data = [list(map(int, x)) for x in data]

    def check_row(row: List) -> bool:
        if row[0] > row[1]: #List is decreasing
            for i in range(len(row) - 1):
                diff = row[i] - row[i+1]
                if diff <= 0 or diff > 3:
                    return False
        elif row[1] > row[0]: 
            for i in range(len(row) - 1):
                diff = row[i+1] - row[i]
                if diff <= 0 or diff > 3:
                    return False                
        else:
            return False
        return True

    valids = [check_row(row) for row in data]

    for i, status in enumerate(valids):
        if not status: #status == False
            report = data[i]
            for j in range(len(report)):
                one_omitted = report[:j] + report[j+1:]
                if check_row(one_omitted):
                    valids[i] = True
    
            
        
    return sum(valids)

part2(data)


Calling part2: Memory used 1105920 kB; Execution Time: 0.013530708383768797 s


296